# Preparing the data
Combining data into two datasets, one that contains all rent prices and one that contains all buy prices.

In [1]:
import pandas as pd
import glob
import os
buy_files = glob.glob(os.path.join('./dataset/', 'apartments_pl*'))
rent_files = glob.glob(os.path.join('./dataset/', 'apartments_rent*'))

buy_df = pd.DataFrame()
rent_df = pd.DataFrame()

for file in buy_files:
    df = pd.read_csv(file)
    base = os.path.basename(file)
    date = base[-11:-4] # 2024_02 transform later if needed
    df['date']=date
    buy_df = pd.concat([buy_df, df])

for file in rent_files:
    df = pd.read_csv(file)
    base = os.path.basename(file)
    date = base[-11:-4] # 2024_02 transform later if needed
    df['date']=date
    rent_df = pd.concat([rent_df, df])

buy_df.to_csv('buy.csv', index=False)
rent_df.to_csv('rent.csv', index=False)
